In [1]:
import random
import numpy as np
import pandas as pd
import torch
from data import MyData
from torch.utils.data import DataLoader
from model import BaseLSTM,MyModel,MyLSTM
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mycolors = plt.cm.RdBu(list(range(0,256,256//10)))

def setup_seed(seed):
	torch.manual_seed(seed)
	torch.cuda.manual_seed_all(seed)
	np.random.seed(seed)
	random.seed(seed)

C:\Users\Tom\anaconda3\envs\main\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
testdata = MyData(data_path='./data/TestData.json', l=4, frac=1)
testdata.get_xpredict_sequence(x=8,update=True)
(len(testdata)*8)

加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 2882 个序列


23056

In [3]:
RMSE = {}

def Predict(model=BaseLSTM().to(device), model_name='24h',t=1):

    model.load_state_dict(torch.load(f'checkpoints/{model_name}/best.pth'))
    testdata = MyData(data_path='./data/TestData.json', l=4, frac=1)
    testdata.get_xpredict_sequence(x=t,update=True)
    batch_size = 4096
    test_dataloader = DataLoader(testdata, batch_size=batch_size, shuffle=False)

    GT_Predict = pd.DataFrame(np.full([len(testdata)*t, 5], np.nan),
                              columns=['True_lat', 'True_lon', 'Predict_lat', 'Predict_lon', 'SE'])

    for i, batch in enumerate(test_dataloader):
        Predict_y = []
        X, y, _, _ = batch
        X = X.to(device)
        for j in range(t):
            predict_y = model(X)
            Predict_y.append(torch.unsqueeze(predict_y,dim=0))
            X = torch.cat([X[:,1:,:], torch.unsqueeze(predict_y,dim=1)], dim=1)
        Predict_y = torch.cat(Predict_y, dim=0)
        Predict_y = Predict_y.permute(1, 0, 2)
        Predict_y = torch.reshape(Predict_y, (-1,4))
        Predict_y = testdata.inverse_norm(Predict_y.cpu().detach()).numpy()
        true_y = testdata.inverse_norm(y.view(-1,4)).numpy()
    GT_Predict.iloc[:, :2] = true_y[:, :2] * 0.1

    GT_Predict.iloc[:, 2:4] = Predict_y[:, :2] * 0.1
    GT_Predict.iloc[:, 4] = np.sum(np.square(GT_Predict.iloc[:, :2].values - GT_Predict.iloc[:, 2:4].values), axis=1)

    GT_Predict.to_csv(f'checkpoints/{model_name}/TestPredict.csv')
    RMSE[f'{model_name}-{t*6}h'] = np.sqrt(np.mean(GT_Predict['SE'].values))
    print(RMSE)

def test_19(model,model_name):
    testdata = MyData(data_path='./data/TestData.json', l=4, frac=1, TaifengID=19)
    batch_size = 4096
    GT_Predict = pd.DataFrame(np.full([len(testdata), 5], np.nan),
	                          columns=['True_lat', 'True_lon', 'Predict_lat', 'Predict_lon', 'SE'])
    test_dataloader = DataLoader(testdata, batch_size=batch_size, shuffle=False)
    for i, batch in enumerate(test_dataloader):
        X, y, _, _ = batch
        model.eval()
        predict_y = testdata.inverse_norm(model(X.to(device)).cpu().detach()).numpy()
        true_y = testdata.inverse_norm(y).numpy()
    GT_Predict.iloc[:, :2] = true_y[:, :2] * 0.1
    GT_Predict.iloc[:, 2:4] = predict_y[:, :2] * 0.1
    GT_Predict.iloc[:, 4] = np.sum(np.square(GT_Predict.iloc[:, :2].values - GT_Predict.iloc[:, 2:4].values), axis=1)
    GT_Predict.to_csv(f'checkpoints/{model_name}/{model_name}_19Taifeng.csv')

In [ ]:
from model import BaseLSTM,MyModel,MyLSTM,BPNetModel,MyChallenge,MyTransformer
model = MyChallenge().to(device)
model_name = 'BaseTransformer'
model.load_state_dict(torch.load(f'checkpoints/{model_name}/best.pth'))
test_19(model, model_name)
Predict(model=model,model_name=model_name,t=1)
Predict(model=model,model_name=model_name,t=4)
Predict(model=model,model_name=model_name,t=8)
Predict(model=model,model_name=model_name,t=12)

In [4]:
from model import BaseLSTM,MyModel,MyLSTM,BPNetModel,MyChallenge
model = MyChallenge().to(device)
model_name = 'BaseChallenge'
model.load_state_dict(torch.load(f'checkpoints/{model_name}/best.pth'))
test_19(model, model_name)
Predict(model=model,model_name=model_name,t=1)
Predict(model=model,model_name=model_name,t=4)
Predict(model=model,model_name=model_name,t=8)
Predict(model=model,model_name=model_name,t=12)

加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 42 个序列
加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 3931 个序列
{'BaseChallenge-6h': 0.6248667384541925}
加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 3475 个序列
{'BaseChallenge-6h': 0.6248667384541925, 'BaseChallenge-24h': 2.0119720794956835}
加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 2882 个序列
{'BaseChallenge-6h': 0.6248667384541925, 'BaseChallenge-24h': 2.0119720794956835, 'BaseChallenge-48h': 4.645990635209887}
加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 2333 个序列
{'BaseChallenge-6h': 0.6248667384541925, 'BaseChallenge-24h': 2.0119720794956835, 'BaseChallenge-48h': 4.645990635209887, 'BaseChallenge-72h': 7.594463086493933}


In [8]:
from model import BaseLSTM,MyModel,MyLSTM,BPNetModel
model = BPNetModel().to(device)
model_name = 'BaseMLP'
model.load_state_dict(torch.load(f'checkpoints/{model_name}/best.pth'))
test_19(model, model_name)

加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 42 个序列


In [5]:
Predict(model=model,model_name=model_name,t=1)
Predict(model=model,model_name=model_name,t=4)
Predict(model=model,model_name=model_name,t=8)
Predict(model=model,model_name=model_name,t=12)

加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 3931 个序列
{'BaseMLP-6h': 0.8494742209863915}
加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 3475 个序列
{'BaseMLP-6h': 0.8494742209863915, 'BaseMLP-24h': 2.3737057335599894}
加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 2882 个序列
{'BaseMLP-6h': 0.8494742209863915, 'BaseMLP-24h': 2.3737057335599894, 'BaseMLP-48h': 5.142441222774898}
加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 2333 个序列
{'BaseMLP-6h': 0.8494742209863915, 'BaseMLP-24h': 2.3737057335599894, 'BaseMLP-48h': 5.142441222774898, 'BaseMLP-72h': 8.543561952304104}


In [5]:
traindata = MyData(data_path='./data/TrainData.json', frac=1)

加载数据，总共有 436 个台风， 12839 条台风数据
总共生成 225792 个序列


In [9]:
from model import BaseLSTM,MyModel,MyLSTM,BPNetModel,MyChallenge
model = MyLSTM().to(device)
model_name = '2LayerBidirLSTM'
model.load_state_dict(torch.load(f'checkpoints/{model_name}/best.pth'))
test_19(model, model_name)
Predict(model=model,model_name=model_name,t=1)
Predict(model=model,model_name=model_name,t=4)
Predict(model=model,model_name=model_name,t=8)
Predict(model=model,model_name=model_name,t=12)

加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 42 个序列
加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 3931 个序列
{'2LayerBidirLSTM-6h': 0.657032963493768}
加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 3475 个序列
{'2LayerBidirLSTM-6h': 0.657032963493768, '2LayerBidirLSTM-24h': 2.555664242575094}
加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 2882 个序列
{'2LayerBidirLSTM-6h': 0.657032963493768, '2LayerBidirLSTM-24h': 2.555664242575094, '2LayerBidirLSTM-48h': 10.410849471434586}
加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 2333 个序列
{'2LayerBidirLSTM-6h': 0.657032963493768, '2LayerBidirLSTM-24h': 2.555664242575094, '2LayerBidirLSTM-48h': 10.410849471434586, '2LayerBidirLSTM-72h': 22.467086413380407}


In [10]:
from model import BaseLSTM,MyModel,MyLSTM,BPNetModel,MyChallenge,MyTransformer,MyBidirLSTM
model = MyBidirLSTM().to(device)
model_name = 'BidirLSTM'
model.load_state_dict(torch.load(f'checkpoints/{model_name}/best.pth'))
test_19(model, model_name)
Predict(model=model,model_name=model_name,t=1)
Predict(model=model,model_name=model_name,t=4)
Predict(model=model,model_name=model_name,t=8)
Predict(model=model,model_name=model_name,t=12)

加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 42 个序列
加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 3931 个序列
{'2LayerBidirLSTM-6h': 0.657032963493768, '2LayerBidirLSTM-24h': 2.555664242575094, '2LayerBidirLSTM-48h': 10.410849471434586, '2LayerBidirLSTM-72h': 22.467086413380407, 'BidirLSTM-6h': 0.6227254850720666}
加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 3475 个序列
{'2LayerBidirLSTM-6h': 0.657032963493768, '2LayerBidirLSTM-24h': 2.555664242575094, '2LayerBidirLSTM-48h': 10.410849471434586, '2LayerBidirLSTM-72h': 22.467086413380407, 'BidirLSTM-6h': 0.6227254850720666, 'BidirLSTM-24h': 2.0193339704721387}
加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 2882 个序列
{'2LayerBidirLSTM-6h': 0.657032963493768, '2LayerBidirLSTM-24h': 2.555664242575094, '2LayerBidirLSTM-48h': 10.410849471434586, '2LayerBidirLSTM-72h': 22.467086413380407, 'BidirLSTM-6h': 0.6227254850720666, 'BidirLSTM-24h': 2.0193339704721387, 'BidirLSTM-48h': 4.578090537003167}
加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 2333 个序列
{'2

In [11]:
from model import BaseLSTM,MyModel,MyLSTM,BPNetModel,MyChallenge,MyTransformer,MyL2Layer
model = MyL2Layer().to(device)
model_name = '2LayerLSTM'
model.load_state_dict(torch.load(f'checkpoints/{model_name}/best.pth'))
test_19(model, model_name)
Predict(model=model,model_name=model_name,t=1)
Predict(model=model,model_name=model_name,t=4)
Predict(model=model,model_name=model_name,t=8)
Predict(model=model,model_name=model_name,t=12)

加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 42 个序列
加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 3931 个序列
{'2LayerBidirLSTM-6h': 0.657032963493768, '2LayerBidirLSTM-24h': 2.555664242575094, '2LayerBidirLSTM-48h': 10.410849471434586, '2LayerBidirLSTM-72h': 22.467086413380407, 'BidirLSTM-6h': 0.6227254850720666, 'BidirLSTM-24h': 2.0193339704721387, 'BidirLSTM-48h': 4.578090537003167, 'BidirLSTM-72h': 7.3484993442082525, '2LayerLSTM-6h': 0.5703176626571449}
加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 3475 个序列
{'2LayerBidirLSTM-6h': 0.657032963493768, '2LayerBidirLSTM-24h': 2.555664242575094, '2LayerBidirLSTM-48h': 10.410849471434586, '2LayerBidirLSTM-72h': 22.467086413380407, 'BidirLSTM-6h': 0.6227254850720666, 'BidirLSTM-24h': 2.0193339704721387, 'BidirLSTM-48h': 4.578090537003167, 'BidirLSTM-72h': 7.3484993442082525, '2LayerLSTM-6h': 0.5703176626571449, '2LayerLSTM-24h': 1.9101852364611167}
加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 2882 个序列
{'2LayerBidirLSTM-6h': 0.657032963493768, '2

In [4]:
from model import BaseLSTM,MyModel,MyLSTM,BPNetModel,MyChallenge,MyTransformer,MyL2Layer
model = MyTransformer().to(device)
model_name = 'BaseTransformer'
model.load_state_dict(torch.load(f'checkpoints/{model_name}/best.pth'))
test_19(model, model_name)
Predict(model=model,model_name=model_name,t=1)
Predict(model=model,model_name=model_name,t=4)
Predict(model=model,model_name=model_name,t=8)
Predict(model=model,model_name=model_name,t=12)

加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 42 个序列
加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 3931 个序列
{'BaseTransformer-6h': 9.272265216044849}
加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 3475 个序列
{'BaseTransformer-6h': 9.272265216044849, 'BaseTransformer-24h': 9.53510725410233}
加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 2882 个序列
{'BaseTransformer-6h': 9.272265216044849, 'BaseTransformer-24h': 9.53510725410233, 'BaseTransformer-48h': 10.476215067267203}
加载数据，总共有 152 个台风， 4539 条台风数据
总共生成 3931 个序列
总共生成 2333 个序列
{'BaseTransformer-6h': 9.272265216044849, 'BaseTransformer-24h': 9.53510725410233, 'BaseTransformer-48h': 10.476215067267203, 'BaseTransformer-72h': 11.743448168884523}


In [28]:
a

array([[178,   2,   5],
       [402,  32,   5],
       [ 17,  23,   5],
       ...,
       [330,   8,   5],
       [ 64,  39,   5],
       [406,   8,   5]])